## Cleaning Dataframe

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../../data/processed/total.csv')

In [3]:
data.head()

,Unnamed: 0,year,playername,team,age,attempts,yards_run,tds_run,longgain_run,yardsperatt,...,catchpercent,yards_rec,yardsperrec,tds_rec,firstdowns,longgain_rec,yardspertarget,recpergame,yardspergame_rec,fumbles
0,0,2020,Aaron Jones,GNB,26,75,389,5,75,5.2,...,73.3%,261,7.9,2,12,30,5.8,3.7,29.0,2.0
1,1,2019,Aaron Jones,GNB,25,236,1084,16,56,4.6,...,72.1%,474,9.7,3,18,67,7.0,3.1,29.6,3.0
2,2,2018,Aaron Jones,GNB,24,133,728,8,67,5.5,...,74.3%,206,7.9,1,12,24,5.9,2.2,17.2,1.0
3,3,2017,Aaron Jones,GNB,23,81,448,4,46,5.5,...,50.0%,22,2.4,0,2,9,1.2,0.8,1.8,0.0
4,4,2020,Adrian Peterson,DET,35,80,314,2,27,3.9,...,68.8%,78,7.1,0,3,18,4.9,1.0,7.1,0.0


In [4]:
data.columns

Index(['Unnamed: 0', 'year', 'playername', 'team', 'age', 'attempts',
       'yards_run', 'tds_run', 'longgain_run', 'yardsperatt',
       'yardspergame_run', 'basesalarycap (10^8)', 'Percenthit (%)',
       'caphit (M)', 'cashspent (M)', 'g', 'gs', 'tgt', 'rec', 'catchpercent',
       'yards_rec', 'yardsperrec', 'tds_rec', 'firstdowns', 'longgain_rec',
       'yardspertarget', 'recpergame', 'yardspergame_rec', 'fumbles'],
      dtype='object')

In [5]:
data = data.drop('Unnamed: 0',axis=1)
data = data.drop('playername', axis=1)
data = data.drop('team', axis=1)
data = data.drop('basesalarycap (10^8)',axis=1)
data = data.drop('cashspent (M)',axis=1)
data = data.drop('caphit (M)',axis=1)
data = data.drop('catchpercent',axis=1)
data = data.drop('longgain_run',axis=1)
data = data.drop('longgain_rec',axis=1)

In [6]:
y_val = data['Percenthit (%)']

In [7]:
x_data = data.drop('Percenthit (%)',axis=1)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3)

## Scaling data

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
scaler.fit(x_train)

/home/brian/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))

In [13]:
x_train = pd.DataFrame(data=scaler.transform(x_train),
                       columns=x_train.columns,index=x_train.index)
x_test = pd.DataFrame(data=scaler.transform(x_test),
                      columns=x_test.columns,index=x_test.index)

## Creating Featuring Columns

In [14]:
data.columns

Index(['year', 'age', 'attempts', 'yards_run', 'tds_run', 'yardsperatt',
       'yardspergame_run', 'Percenthit (%)', 'g', 'gs', 'tgt', 'rec',
       'yards_rec', 'yardsperrec', 'tds_rec', 'firstdowns', 'yardspertarget',
       'recpergame', 'yardspergame_rec', 'fumbles'],
      dtype='object')

In [15]:
import tensorflow as tf

In [16]:
year = tf.feature_column.numeric_column('year')
age = tf.feature_column.numeric_column('age')
attempts = tf.feature_column.numeric_column('attempts')
runyards = tf.feature_column.numeric_column('yards_run')
runtds = tf.feature_column.numeric_column('tds_run')
ypa = tf.feature_column.numeric_column('yardsperatt')
ypgrun = tf.feature_column.numeric_column('yardspergame_run')
games = tf.feature_column.numeric_column('g')
started = tf.feature_column.numeric_column('gs')
targets = tf.feature_column.numeric_column('tgt')
receptions = tf.feature_column.numeric_column('rec')
recyards = tf.feature_column.numeric_column('yards_rec')
ypr = tf.feature_column.numeric_column('yardsperrec')
rectds = tf.feature_column.numeric_column('tds_rec')
firstdown = tf.feature_column.numeric_column('firstdowns')
ypt = tf.feature_column.numeric_column('yardspertarget')
rpg = tf.feature_column.numeric_column('recpergame')
ypgrec = tf.feature_column.numeric_column('yardspergame_rec')
fum = tf.feature_column.numeric_column('fumbles')

In [17]:
feat_cols = [year,age,attempts,runyards,runtds,ypa,ypgrun,games,started,
            targets,receptions,recyards,ypr,rectds,firstdown,ypt,rpg,ypgrec,fum]

In [18]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,
                                                batch_size=10,
                                                num_epochs=1000,
                                                shuffle=True)

In [19]:
model = tf.estimator.DNNRegressor(hidden_units=[10,10,10],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/tmp/tmpaopdimuf', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_tf_random_seed': 1}


In [20]:
model.train(input_fn=input_func,steps=20000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpaopdimuf/model.ckpt.
INFO:tensorflow:loss = 10.986355, step = 1
INFO:tensorflow:global_step/sec: 596.813
INFO:tensorflow:loss = 39.28942, step = 101 (0.170 sec)
INFO:tensorflow:global_step/sec: 628.763
INFO:tensorflow:loss = 10.460766, step = 201 (0.161 sec)
INFO:tensorflow:global_step/sec: 594.819
INFO:tensorflow:loss = 7.9094567, step = 301 (0.166 sec)
INFO:tensorflow:global_step/sec: 615.808
INFO:tensorflow:loss = 18.410315, step = 401 (0.162 sec)
INFO:tensorflow:global_step/sec: 653.617
INFO:tensorflow:loss = 18.987411, step = 501 (0.155 sec)
INFO:tensorflow:global_step/sec: 645.56
INFO:tensorflow:loss = 8.853857, step = 601 (0.154 sec)
INFO:tensorflow:global_step/sec: 631.257
INFO:tensorflow:loss = 58.150024, step = 701 (0.160 sec)
INFO:tensorflow:global_step/sec: 669.221
INFO:tensorflow:loss = 7.612443, step = 801 (0.146 sec)
INFO:tensorflow:global_step/sec: 635.161
INFO:tensorflow:l

INFO:tensorflow:global_step/sec: 607.776
INFO:tensorflow:loss = 23.68899, step = 8301 (0.164 sec)
INFO:tensorflow:global_step/sec: 581.931
INFO:tensorflow:loss = 4.4814854, step = 8401 (0.172 sec)
INFO:tensorflow:global_step/sec: 672.116
INFO:tensorflow:loss = 42.231922, step = 8501 (0.149 sec)
INFO:tensorflow:global_step/sec: 591.909
INFO:tensorflow:loss = 8.367837, step = 8601 (0.169 sec)
INFO:tensorflow:global_step/sec: 674.438
INFO:tensorflow:loss = 11.907189, step = 8701 (0.148 sec)
INFO:tensorflow:global_step/sec: 662.29
INFO:tensorflow:loss = 9.331015, step = 8801 (0.152 sec)
INFO:tensorflow:global_step/sec: 608.509
INFO:tensorflow:loss = 7.706485, step = 8901 (0.164 sec)
INFO:tensorflow:global_step/sec: 587.118
INFO:tensorflow:loss = 8.077607, step = 9001 (0.170 sec)
INFO:tensorflow:global_step/sec: 591.013
INFO:tensorflow:loss = 12.220795, step = 9101 (0.169 sec)
INFO:tensorflow:global_step/sec: 639.23
INFO:tensorflow:loss = 6.252752, step = 9201 (0.156 sec)
INFO:tensorflow:gl

INFO:tensorflow:global_step/sec: 636.717
INFO:tensorflow:loss = 11.635642, step = 16601 (0.156 sec)
INFO:tensorflow:global_step/sec: 589.568
INFO:tensorflow:loss = 10.001309, step = 16701 (0.171 sec)
INFO:tensorflow:global_step/sec: 577.312
INFO:tensorflow:loss = 7.9859996, step = 16801 (0.173 sec)
INFO:tensorflow:global_step/sec: 670.815
INFO:tensorflow:loss = 8.387409, step = 16901 (0.149 sec)
INFO:tensorflow:global_step/sec: 638.083
INFO:tensorflow:loss = 20.187119, step = 17001 (0.159 sec)
INFO:tensorflow:global_step/sec: 675.552
INFO:tensorflow:loss = 3.06216, step = 17101 (0.147 sec)
INFO:tensorflow:global_step/sec: 641.812
INFO:tensorflow:loss = 8.707544, step = 17201 (0.154 sec)
INFO:tensorflow:global_step/sec: 700.409
INFO:tensorflow:loss = 3.0017915, step = 17301 (0.143 sec)
INFO:tensorflow:global_step/sec: 552.562
INFO:tensorflow:loss = 4.169988, step = 17401 (0.180 sec)
INFO:tensorflow:global_step/sec: 646.343
INFO:tensorflow:loss = 26.732555, step = 17501 (0.155 sec)
INFO:

In [21]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,
                                                        batch_size=10,
                                                        num_epochs=1,
                                                        shuffle=False)

In [22]:
pred_gen = model.predict(predict_input_func)

In [23]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from /tmp/tmpaopdimuf/model.ckpt-20000


In [24]:
predictions

[{'predictions': array([6.526651], dtype=float32)},
 {'predictions': array([1.40039], dtype=float32)},
 {'predictions': array([0.6763861], dtype=float32)},
 {'predictions': array([0.6753453], dtype=float32)},
 {'predictions': array([3.4836023], dtype=float32)},
 {'predictions': array([0.92336804], dtype=float32)},
 {'predictions': array([0.41198325], dtype=float32)},
 {'predictions': array([1.1267512], dtype=float32)},
 {'predictions': array([0.77748096], dtype=float32)},
 {'predictions': array([1.2345377], dtype=float32)},
 {'predictions': array([0.6171829], dtype=float32)},
 {'predictions': array([1.071959], dtype=float32)},
 {'predictions': array([0.33873683], dtype=float32)},
 {'predictions': array([0.40122503], dtype=float32)},
 {'predictions': array([2.3143406], dtype=float32)},
 {'predictions': array([0.42105055], dtype=float32)},
 {'predictions': array([1.3166249], dtype=float32)},
 {'predictions': array([1.1308435], dtype=float32)},
 {'predictions': array([1.9839766], dtype=fl

In [25]:
final_preds = []

for pred in predictions:
    final_preds.append(pred['predictions'])

In [26]:
from sklearn.metrics import mean_squared_error

In [27]:
mean_squared_error(y_test,final_preds)**0.5

1.5501570942865854

In [28]:
final_preds

[array([6.526651], dtype=float32),
 array([1.40039], dtype=float32),
 array([0.6763861], dtype=float32),
 array([0.6753453], dtype=float32),
 array([3.4836023], dtype=float32),
 array([0.92336804], dtype=float32),
 array([0.41198325], dtype=float32),
 array([1.1267512], dtype=float32),
 array([0.77748096], dtype=float32),
 array([1.2345377], dtype=float32),
 array([0.6171829], dtype=float32),
 array([1.071959], dtype=float32),
 array([0.33873683], dtype=float32),
 array([0.40122503], dtype=float32),
 array([2.3143406], dtype=float32),
 array([0.42105055], dtype=float32),
 array([1.3166249], dtype=float32),
 array([1.1308435], dtype=float32),
 array([1.9839766], dtype=float32),
 array([0.8059934], dtype=float32),
 array([4.082472], dtype=float32),
 array([0.41978407], dtype=float32),
 array([0.7151649], dtype=float32),
 array([0.5624677], dtype=float32),
 array([0.54626346], dtype=float32),
 array([0.79483944], dtype=float32),
 array([0.43027264], dtype=float32),
 array([0.9445199], dty

In [29]:
pred_total = 0
test_total = 0

for pred in final_preds:
    pred_total = pred_total + pred[0]
    
for test in y_test.values:
    test_total = test_total + test

In [30]:
y_test_values = y_test.values
total_error = 0
percent_error = 0

for i in range(0,len(final_preds)):
    true_error = final_preds[i][0] - y_test_values[i] 
    print(true_error)
    total_error = total_error + true_error
    percent_error = percent_error + ((true_error /
                                     y_test_values[i]) / len(final_preds))


0.33384110168756376
0.0320065778940164
-1.3463182211162823
0.26902475986739055
3.004654381193515
0.49864166858777476
-0.739238552939623
-1.896622799276336
0.46447282981097215
-0.29462894598642997
-1.027329284679599
0.25709784489237697
-0.15145924628949625
-0.5194556284513281
1.705997704846005
-0.048128058913738114
-1.311966499473309
0.054658418584354385
1.4042750987949226
-1.2626241798598161
-4.190673519917608
0.06077875556499884
-1.197693761172887
-1.8003371837662483
0.0860644513694801
-0.2678598143342452
0.05901515376781985
0.5793776010110127
0.030035077555095713
-0.9158567126883532
0.16457377456872924
0.3522825965921322
1.5251232297463342
0.16051067437582867
-0.8015307152342652
-0.18139279633253624
0.3781166241394347
-0.9474812293411194
1.332596554057509
0.42241790496145903
0.027476394091286105
-6.686089311739647
-0.4364373428755689
-1.5420891487380954
0.020661560957588587
0.6489982945578439
0.0261880976489956
-0.5264701871793864
-0.016483361513835337
-2.8324180801749588
0.386519634

In [31]:
print("combined error: ")
print(total_error)
print("combined error / # of entries: ")
print(total_error/(len(final_preds)))
print("percent error: ")
print(percent_error)

combined error: 
-63.70184320399859
combined error / # of entries: 
-0.3519438851049646
percent error: 
0.4060653740628738


In [32]:
from sklearn.metrics import explained_variance_score

In [33]:
explained_variance_score(y_test, final_preds)

0.45740430280270306